# CDF Response Time Analysis with Varying Job Populations

This example demonstrates how CDF response time analysis changes with different job populations in a closed network. It compares response time distributions for various population sizes.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.fluid.*
import jline.util.matrix.*

In [ ]:
// CDF Response Time Analysis Example 5
// Closed network with varying job populations - CDF comparison

val nJobsArray = intArrayOf(1, 4, 8)
val labels = mutableListOf<String>()

// Storage for results across different population sizes
val avgRespTResults = mutableMapOf<Int, Matrix>()
val avgRespTFromCDFResults = mutableMapOf<Int, Matrix>()
val powerMoment2Results = mutableMapOf<Int, Matrix>()
val varianceResults = mutableMapOf<Int, Matrix>()
val sqCoeffOfVariationResults = mutableMapOf<Int, Matrix>()
val cdfResults = mutableMapOf<Int, Array<Array<Matrix?>>>()

In [ ]:
for ((ni, N) in nJobsArray.withIndex()) {
    println("Processing N = $N jobs...")
    
    // Create model for this population size
    val model = Network("model")
    
    // Create closed queueing network
    val delay = Delay(model, "Delay")
    val queue1 = Queue(model, "Queue1", SchedStrategy.PS)
    val queue2 = Queue(model, "Queue2", SchedStrategy.PS)
    
    // Single closed class with N jobs
    val jobClass = ClosedClass(model, "Class1", N, delay, 0)
    
    // Class doesn't complete (stays in system)
    jobClass.completes = false
    
    // Service processes - all exponential
    delay.setService(jobClass, Exp(1.0))     // rate = 1
    queue1.setService(jobClass, Exp(0.5))    // rate = 0.5
    queue2.setService(jobClass, Exp(0.5))    // rate = 0.5
    
    // Circular routing through all 3 stations
    val P = model.initRoutingMatrix()
    P.set(jobClass, jobClass, delay, queue1, 1.0)
    P.set(jobClass, jobClass, queue1, queue2, 1.0)
    P.set(jobClass, jobClass, queue2, delay, 1.0)
    model.link(P)
    
    labels.add("N=$N jobs")
    
    println("Model created for N=$N, proceeding to solve...")
}

In [ ]:
// Solve for each population size
for ((ni, N) in nJobsArray.withIndex()) {
    println("\nSolving for N = $N jobs...")
    
    // Recreate model (since we need fresh instances)
    val model = Network("model")
    val delay = Delay(model, "Delay")
    val queue1 = Queue(model, "Queue1", SchedStrategy.PS)
    val queue2 = Queue(model, "Queue2", SchedStrategy.PS)
    val jobClass = ClosedClass(model, "Class1", N, delay, 0)
    jobClass.completes = false
    
    delay.setService(jobClass, Exp(1.0))
    queue1.setService(jobClass, Exp(0.5))
    queue2.setService(jobClass, Exp(0.5))
    
    val P = model.initRoutingMatrix()
    P.set(jobClass, jobClass, delay, queue1, 1.0)
    P.set(jobClass, jobClass, queue1, queue2, 1.0)
    P.set(jobClass, jobClass, queue2, delay, 1.0)
    model.link(P)
    
    // Solve with Fluid solver
    val fluidOptions = FLD.defaultOptions()
    fluidOptions.iterMax = 100
    val solver = FLD(model, fluidOptions)
    
    avgRespTResults[ni] = solver.avgRespT
    val fc = solver.cdfRespT
    cdfResults[ni] = fc
    
    // Calculate statistics from CDF
    val avgRespTFromCDF = Matrix.zeros(model.numberOfStations, model.numberOfClasses)
    val powerMoment2 = Matrix.zeros(model.numberOfStations, model.numberOfClasses)
    val variance = Matrix.zeros(model.numberOfStations, model.numberOfClasses)
    val sqCoeffOfVariation = Matrix.zeros(model.numberOfStations, model.numberOfClasses)
    
    for (c in 0 until model.numberOfClasses) {
        for (i in 0 until model.numberOfStations) {
            if (fc[i][c] != null && fc[i][c].length() > 1) {
                val cdfData = fc[i][c]
                var avgSum = 0.0
                var moment2Sum = 0.0
                
                // Calculate mean and second moment from CDF
                for (j in 1 until cdfData.length()) {
                    val diff = cdfData.get(j, 0) - cdfData.get(j-1, 0)
                    val value = cdfData.get(j, 1)
                    avgSum += diff * value
                    moment2Sum += diff * (value * value)
                }
                
                avgRespTFromCDF.set(i, c, avgSum)
                powerMoment2.set(i, c, moment2Sum)
                
                val varianceValue = moment2Sum - (avgSum * avgSum)
                variance.set(i, c, varianceValue)
                
                if (avgSum > 0) {
                    sqCoeffOfVariation.set(i, c, varianceValue / (avgSum * avgSum))
                }
            }
        }
    }
    
    avgRespTFromCDFResults[ni] = avgRespTFromCDF
    powerMoment2Results[ni] = powerMoment2
    varianceResults[ni] = variance
    sqCoeffOfVariationResults[ni] = sqCoeffOfVariation
    
    println("Average Response Time from CDF for N=$N:")
    avgRespTFromCDF.print()
}

In [ ]:
// Display CDF comparison results for station 2 (queue2)
val stationToAnalyze = 2  // Station 2 (Queue2)
println("\n=== CDF Analysis Results for Station $stationToAnalyze (Queue2) ===")

for ((ni, N) in nJobsArray.withIndex()) {
    println("\nN = $N jobs:")
    
    val fc = cdfResults[ni]
    if (fc != null) {
        for (c in 0 until fc[stationToAnalyze].size) {
            val cdfData = fc[stationToAnalyze][c]
            if (cdfData != null && cdfData.length() > 0) {
                println("  Class $c:")
                println("    CDF points: ${cdfData.length()}")
                println("    Response time range: [${cdfData.get(0, 1)}, ${cdfData.get(cdfData.length()-1, 1)}]")
                
                // Show percentile values
                val n = cdfData.length()
                if (n > 10) {
                    val idx50 = (n * 0.5).toInt()
                    val idx90 = (n * 0.9).toInt()
                    val idx95 = (n * 0.95).toInt()
                    
                    println("    50th percentile: ${cdfData.get(idx50, 1)}")
                    println("    90th percentile: ${cdfData.get(idx90, 1)}")
                    println("    95th percentile: ${cdfData.get(idx95, 1)}")
                }
                
                // Show statistics
                val avgFromCDF = avgRespTFromCDFResults[ni]?.get(stationToAnalyze, c) ?: 0.0
                val varianceValue = varianceResults[ni]?.get(stationToAnalyze, c) ?: 0.0
                val scv = sqCoeffOfVariationResults[ni]?.get(stationToAnalyze, c) ?: 0.0
                
                println("    Mean response time: $avgFromCDF")
                println("    Variance: $varianceValue")
                println("    Squared coeff. of variation: $scv")
            }
        }
    }
}

In [ ]:
// Summary comparison across populations
println("\n=== Summary: Response Time Statistics Across Populations ===")
println("Station\tPopulation\tMean RespT\tVariance\tSCV")
println("------------------------------------------------------")

for (i in 0 until 3) {  // All stations
    for ((ni, N) in nJobsArray.withIndex()) {
        val avgFromCDF = avgRespTFromCDFResults[ni]?.get(i, 0) ?: 0.0
        val varianceValue = varianceResults[ni]?.get(i, 0) ?: 0.0
        val scv = sqCoeffOfVariationResults[ni]?.get(i, 0) ?: 0.0
        
        println("$i\t$N\t\t%.3f\t\t%.3f\t\t%.3f".format(avgFromCDF, varianceValue, scv))
    }
    if (i < 2) println("------------------------------------------------------")
}

println("\nObservations:")
println("- As population increases, mean response times generally increase")
println("- Variance and SCV patterns show the impact of congestion")
println("- Station 2 (Queue2) shows the most sensitivity to population changes")

This example demonstrates:

1. **Population Impact Analysis**: 
   - Compares response time CDFs across different job populations (1, 4, 8 jobs)
   - Shows how congestion affects response time distributions
2. **Closed Network Dynamics**:
   - Circular routing through Delay → Queue1 → Queue2 → Delay
   - Non-completing jobs (jobs circulate indefinitely)
3. **Statistical Analysis from CDFs**:
   - Mean response times computed from CDFs
   - Second moments, variances, and squared coefficients of variation
   - Percentile analysis (50th, 90th, 95th percentiles)
4. **Performance Insights**:
   - Higher populations lead to increased response times
   - Variance and variability patterns reveal congestion effects
   - Different stations show varying sensitivity to population changes

This analysis is valuable for capacity planning and understanding how system performance degrades with increasing load in closed queueing networks.